In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
import lightgbm as lgb

import math

%matplotlib inline
plt.rcParams['font.family'] = 'Hiragino sans'
plt.style.use("ggplot")

In [21]:
# 問6.1
# １日あたりの交通事故死者数の平均が２人であると思われている地区で交通事故死者数を調査した。次の表は100日間の１日あたりの交通事故死者数の度数分布表である

df_1 = pd.DataFrame({
    "0人":1,
    "1人":32,
    "2人":29,
    "3人":12,
    "4人":6,
    '合計':100},
    index = list(range(1,2))
    )
df_1

,0人,1人,2人,3人,4人,合計
1,1,32,29,12,6,100


In [29]:
# (1)１日あたりの交通事故死者数が平均２人のポアソン分布に従っているとき、死者数が0人,1人,2人,3人,4人以上の発生確率を計算しなさい。
λ = 2

def poason(λ,x):
    ans = (math.e**(-λ)*λ**x)/math.factorial(x)
    return ans

p0 = poason(λ,0)
p1 = poason(λ,1)
p2 = poason(λ,2)
p3 = poason(λ,3)
p4_izyou = 1- (p0 + p1 + p2 + p3)

In [30]:
df_ans_1 = pd.DataFrame({
    "0人":p0,
    "1人":p1,
    "2人":p2,
    "3人":p3,
    "4人":p4_izyou,
    '合計':1},
    index = list(range(1,2))
    )
df_ans_1

,0人,1人,2人,3人,4人,合計
1,0.135335,0.270671,0.270671,0.180447,0.142877,1


In [38]:
# (2)一日あたりの交通事故死者数が平均２人のポアソン分布に従っている帰無仮説をたてて、有意水準５％で適合度検定を行う。
# このときの適合度の統計検定量の値はいくらか。またχ^2分布の自由度と棄却域に注意して結論を述べよ

df_2 = pd.DataFrame({
    "0人":[p0,21/100],
    "1人":[p1,32/100],
    "2人":[p2,29/100],
    "3人":[p3,12/100],
    "4人以上":[p4_izyou,6/100]
    },
    index=["母集団","サンプル"])
df_2

,0人,1人,2人,3人,4人以上
母集団,0.135335,0.270671,0.270671,0.180447,0.142877
サンプル,0.210000,0.320000,0.290000,0.120000,0.060000


In [45]:
x_0 = ((p0-21/100)*100)**2/(p0*100)
x_1 = ((p1-32/100)*100)**2/(p1*100)
x_2 = ((p2-29/100)*100)**2/(p2*100)
x_3 = ((p3-12/100)*100)**2/(p3*100)
x_4 = ((p4_izyou-6/100)*100)**2/(p4_izyou*100)
tmp_chisquare = x_0 + x_1 + x_2 + x_3 + x_4
tmp_chisquare

11.988523995302494

In [44]:
# stats.chisuare()で簡単に求められる
from scipy import stats

observed = [21/100 , 32/100 , 29/100 , 12/100 , 6/100]
expected = [p0, p1, p2, p3, p4_izyou]

result = stats.chisquare(observed, expected)
print(result)

Power_divergenceResult(statistic=0.11988523995302493, pvalue=0.998273646450167)


In [48]:
print('セグメントが５つなので、自由度４のχ２分布に従う')
print('有意水準は５％')
print('分布表より、9.49')
print("統計検定量のほうが大きいため、帰無仮説は棄却される。")
print("１日あたりの交通事故死者数は平均2のポアソン分布に従うとは言えない")

セグメントが５つなので、自由度４のχ２分布に従う
有意水準は５％
分布表より、9.49
統計検定量のほうが大きいため、帰無仮説は棄却される。
１日あたりの交通事故死者数は平均2のポアソン分布に従うとは言えない


In [51]:
#(3)1日あたりの交通事故死者数の平均を上の表より推定する。
#その際、「4人以上」は４人とする。一日あたりの交通事故死者数が推定した平均のポアソン分布に従っているとする。
#このときの死者数が0~4人以上の発生確率を計算しなさい

number_of_death = 0*21 + 1*32 + 2*29 + 3*12 + 4*6
mean_nod = number_of_death / 100
print(mean_nod)

p0 = poason(mean_nod,0)
p1 = poason(mean_nod,1)
p2 = poason(mean_nod,2)
p3 = poason(mean_nod,3)
p4_izyou = 1- (p0 + p1 + p2 + p3)

1.5


In [52]:
df_ans_3 = pd.DataFrame({
    "0人":p0,
    "1人":p1,
    "2人":p2,
    "3人":p3,
    "4人":p4_izyou,
    '合計':1},
    index = list(range(1,2))
    )
df_ans_3

,0人,1人,2人,3人,4人,合計
1,0.22313,0.334695,0.251021,0.125511,0.065642,1


In [54]:
# １日あたりの交通事故死者数が（３）でのポアソン分布に従っているという帰無仮説をたて、その有意水準5%で適合度検定を行う。
# このときの適合度検定の検定統計量はいくらか。また自由度や棄却域に注意して結論を述べよ

observed = [21/100 , 32/100 , 29/100 , 12/100 , 6/100]
expected = [p0, p1, p2, p3, p4_izyou]

result = stats.chisquare(observed, expected)
print(result)

Power_divergenceResult(statistic=0.008197414652572035, pvalue=0.9999916232158315)


In [63]:
print("検定統計量が " +str(result[0]) + "であり、自由度３のχ２分布棄却域は7.81以上のため、帰無仮説は受容される")
print("平均1.5のポアソン分布に従わないとは言えない")

# ちなみに自由度がk-2になるのは、そういうものらしい。詳細はp204~205コメントを参照

検定統計量が 0.008197414652572035であり、自由度３のχ２分布棄却域は7.81以上のため、帰無仮説は受容される
平均1.5のポアソン分布に従わないとは言えない


In [69]:
# 問6.2 次の表は２種類の抗生物質を症状が類似している患者に投与した結果の要約である
# この結果から２種類の薬の効果には差があると考えられるか否かを以下の手順で検定する

df_1 = pd.DataFrame({
    "薬剤":["A","B","計"],
    "改善":[52,40,92],
    "不変":[11,17,28],
    "計":[63,57,120]
    },index=[1,2,3])
df_1

,薬剤,改善,不変,計
1,A,52,11,63
2,B,40,17,57
3,計,92,28,120


In [71]:
#(1)２種類の薬の効果に差がなく独立であると仮定するときの期待度数を計算しろ

df_1_e = pd.DataFrame({
    "薬剤":["A","B","計"],
    "改善":[92*63/120,92*57/120,92],
    "不変":[28*63/120,28*57/120,28],
    "計":[63,57,120]
    },index=[1,2,3])
df_1_e

,薬剤,改善,不変,計
1,A,48.3,14.7,63
2,B,43.7,13.3,57
3,計,92.0,28.0,120


In [78]:
# (2)これらの表からχ２統計量の値を求め、有意水準５％で独立性の検定を行い、結論を述べよ

tmp_chisquare = (52-48.3)**2/48.3 + (40-43.7)**2/43.7 + (11-14.7)**2/14.7 + (17-13.3)**2/13.3
tmp_chisquare

2.557324989492365

In [73]:
# パターン１
observed = [52,40,11,17]
expected = [48.3,43.7,14.7,13.3]

result = stats.chisquare(observed, expected)
print(result)

Power_divergenceResult(statistic=2.557324989492365, pvalue=0.46502019088777347)


In [80]:
# パターン2
df = pd.DataFrame([[52,40],
        [11,17]])

stats.chi2_contingency(df,correction=False)

(2.557324989492365,
 0.10978420248422241,
 1,
 array([[48.3, 43.7],
        [14.7, 13.3]]))

In [81]:
print("自由度1のχ２分布棄却域は3.84以上であるため、帰無仮説は受容される")
print("薬に差があるとは言えない")

自由度1のχ２分布棄却域は3.84以上であるため、帰無仮説は受容される
薬に差があるとは言えない
